# 1 | Markov Decision Processes: A Model of Sequential Decision Making

- High-level motivation: sequential decision-making
- The MDP formalism 
    - Mathematical notation *and* Gym as an implementation convention
    - Introduce the Student MDP using diagram and Gym class

![The student MDP](https://github.com/tombewley/one-hour-rl/blob/main/images/student-mdp.png?raw=true "The student MDP")

In [1]:
from mdp import StudentMDP
from agent import Agent
mdp = StudentMDP(verbose=False)
agent = Agent(mdp) 

In [2]:
print(mdp.initial_probs())
mdp.reset()
print(mdp.state)
print(mdp.action_space(mdp.state))
print(mdp.transition_probs(mdp.state, "Study"))

{'Class 1': 1.0, 'Class 2': 0.0, 'Class 3': 0.0, 'Facebook': 0.0, 'Pub': 0.0, 'Pass': 0.0, 'Asleep': 0.0}
Class 1
{'Go on Facebook', 'Study'}
{'Class 2': 1.0}


Run repeatedly... why does this happen?

In [3]:
mdp.step("Study")
print(mdp.state)

Class 2


Non-deterministic transitions

In [4]:
mdp.state = "Pub"
print(mdp.action_space(mdp.state))
print(mdp.transition_probs(mdp.state, "Have a pint"))

{'Have a pint'}
{'Class 1': 0.2, 'Class 2': 0.4, 'Class 3': 0.4}


In [5]:
mdp.state = "Pub"
mdp.step("Have a pint")
print(mdp.state)

Class 3


Default policy shown in images/student-mdp.png

In [6]:
agent.policy = {
    "Class 1":  {"Study": 0.5, "Go on Facebook": 0.5},
    "Class 2":  {"Study": 0.8, "Fall asleep": 0.2},
    "Class 3":  {"Study": 0.6, "Go to the pub": 0.4},
    "Facebook": {"Keep scrolling": 0.9, "Close Facebook": 0.1},
    "Pub":      {"Have a pint": 1.},
    "Pass":     {"Fall asleep": 1.},
    "Asleep":   {"Stay asleep": 1.}
}

In [7]:
print(agent.policy["Class 1"])
print([agent.act("Class 1") for _ in range(20)])

{'Study': 0.5, 'Go on Facebook': 0.5}
['Study', 'Go on Facebook', 'Go on Facebook', 'Go on Facebook', 'Go on Facebook', 'Go on Facebook', 'Study', 'Study', 'Go on Facebook', 'Study', 'Go on Facebook', 'Study', 'Go on Facebook', 'Study', 'Go on Facebook', 'Study', 'Go on Facebook', 'Study', 'Study', 'Study']


In [13]:
mdp.verbose = True
state = mdp.reset()
done = False
while not done:
    state, reward, done, info = mdp.step(agent.act(state))

=========================== EPISODE   7 ===========================
| Time  | State    | Action         | Reward | Next state | Done  |
|-------|----------|----------------|--------|------------|-------|
| 0     | Class 1  | Study          | -2.0   | Class 2    | False |
| 1     | Class 2  | Study          | -2.0   | Class 3    | False |
| 2     | Class 3  | Study          | 10.0   | Pass       | False |
| 3     | Pass     | Fall asleep    |  0.0   | Asleep     | True  |


How "good" is this policy?